# **Import library**

In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import openpyxl

from sklearn import metrics
from sklearn.svm import SVR
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor

import optuna

np.random.seed(123)

%matplotlib inline
plt.rcParams["figure.figsize"] = (16, 8)

In [2]:
import warnings
warnings.simplefilter("ignore")

# ***Data Preparation***

## **CSV Handling**

In [3]:
df = pd.read_excel("training_WeeklyAggregate.xlsx")
df.head()

,sourceid,dstid,dow,mean_travel_time
0,10,241,3,2334.43
1,10,612,5,1529.83
2,10,905,4,1390.04
3,10,407,7,157.91
4,10,603,4,1781.67


In [4]:
test = pd.read_csv("testing_dataset.csv")

## **GeoJson Handling**

In [5]:
london = gpd.read_file("london.json")

In [6]:
london.crs
#"EPSG: 27700" Projected

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
london["MOVEMENT_ID"] = london["MOVEMENT_ID"].astype("int64")
london["x"] = london.centroid.x
london["y"] = london.centroid.y

# ***OpenStreetMap (OSMnx) Initialization***

## Download nodes from OpenStreetMap

In [8]:
G = ox.graph_from_place("London, England", network_type = "drive")

## Important OSMNx Function

In [9]:
G = ox.utils_graph.remove_isolated_nodes(G)
G = ox.speed.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G)

In [10]:
def get_route(dataframe,attr="nodes", weight="length", cpus=1):
    if attr=="nodes":
        rt = ox.shortest_path(G,dataframe["node_id_n_src"],dataframe["node_id_n_dst"], weight=weight,cpus=cpus)
    else:
        rt = ox.shortest_path(G,dataframe["node_id_e_src"],dataframe["node_id_e_dst"], weight=weight,cpus=cpus)
    return rt

In [11]:
def get_attr(route, feature=None):
    return ox.utils_graph.get_route_edge_attributes(G,route,feature)

In [12]:
def get_attr_count(route, attribute):
    attribute_values = []
    for u, v in zip(route[:-1], route[1:]):
        data = min(G.get_edge_data(u, v).values(), key=lambda x: x["length"])
        try:
            attribute_value = data[attribute]
            attribute_values.append(attribute_value)
        except KeyError:
            pass
    return attribute_values

## Extract

In [13]:
# Get osmid from London.json
london["node_id_n"] = ox.distance.nearest_nodes(G,london.centroid.x,london.centroid.y)
london[["node_id_e","nodes_id_2","to_drop"]] = ox.distance.nearest_edges(G,london.centroid.x,london.centroid.y)

# ***Feature Extraction***

## **Get important features from geospatial data**

In [14]:
ldn = london.drop(["DISPLAY_NAME","geometry","msoa_code","msoa_name","la_code","nodes_id_2","to_drop"],axis=1)

src = df.merge(ldn, left_on = "sourceid", right_on = "MOVEMENT_ID", how = "left")
src = src.rename({"geoeast":"geoeast_src", "geonorth":"geonorth_src","popeast":"popeast_src","popnorth":"popnorth_src",
                  "la_name":"la_name_src","area_km2":"area_src", "msoa_code":"msoa_code_src", "x" : "src_x", "y" : "src_y",
                 "node_id_n" : "node_id_n_src", "node_id_e" : "node_id_e_src"},axis = 1)

dst = src.merge(ldn, left_on = "dstid", right_on = "MOVEMENT_ID", how = "left")
dst = dst.rename({"geoeast":"geoeast_dst", "geonorth":"geonorth_dst","popeast":"popeast_dst","popnorth":"popnorth_dst",
                  "la_name":"la_name_dst","area_km2":"area_dst","msoa_code":"msoa_code_dst","x" : "dst_x", "y" : "dst_y",
                  "node_id_n" : "node_id_n_dst", "node_id_e" : "node_id_e_dst"},axis = 1)

df = dst.drop(["MOVEMENT_ID_x","MOVEMENT_ID_y"],axis=1)

df.head()

,sourceid,dstid,dow,mean_travel_time,la_name_src,geoeast_src,geonorth_src,popeast_src,popnorth_src,area_src,...,la_name_dst,geoeast_dst,geonorth_dst,popeast_dst,popnorth_dst,area_dst,dst_x,dst_y,node_id_n_dst,node_id_e_dst
0,10,241,3,2334.43,Newham,542413,182380,542450,182415,0.790802,...,Lambeth,530851,174285,530876,174289,0.584104,-0.118199,51.452418,33776678,33776696
1,10,612,5,1529.83,Newham,542413,182380,542450,182415,0.790802,...,Havering,551347,192721,550534,191931,6.279430,0.184471,51.612990,1138048373,25760609
2,10,905,4,1390.04,Newham,542413,182380,542450,182415,0.790802,...,Southwark,535060,178641,535009,178663,0.662274,-0.056014,51.490606,104397683,270932211
3,10,407,7,157.91,Newham,542413,182380,542450,182415,0.790802,...,Barking and Dagenham,544366,183515,544543,183531,0.796626,0.079911,51.532079,180624488,180624488
4,10,603,4,1781.67,Newham,542413,182380,542450,182415,0.790802,...,Enfield,532416,197315,532432,197391,1.282990,-0.087059,51.659039,11377862,11379331


## **Calculate Displacement**

In [15]:
df["geo_displacement"] = np.linalg.norm(df.loc[:, ["geoeast_src","geonorth_src"]].values - df.loc[:, ["geoeast_dst","geonorth_dst"]], axis=1)
df["geo_displacement_sqrt"] =  np.sqrt(df["geo_displacement"])

## **Calculate Direction**

In [16]:
xDiff = df.loc[:, "geonorth_dst"].values - df.loc[:, "geonorth_src"].values
yDiff = df.loc[:, "geoeast_dst"].values - df.loc[:, "geoeast_src"].values
df["direction"] = np.round(np.degrees(np.arctan2(yDiff,xDiff)))

# **OSMnx features**

In [17]:
df["route_travel_time"] = get_route(df,weight="travel_time")

In [18]:
missing_route_travel_time = df[df["route_travel_time"].isna()]
len(missing_route_travel_time)

32

## Missing route handling

In [19]:
df.loc[missing_route_travel_time.index,"route_travel_time"] = get_route(missing_route_travel_time,"edges","travel_time")

In [20]:
missing_route_travel_time = df[df["route_travel_time"].isna()]
len(missing_route_travel_time)

0

## Extract Features

In [21]:
df["maxspeed_travel_time"] = [round(np.mean(get_attr(row["route_travel_time"], "speed_kph"))) for index, row in df.iterrows()]
df["fastest_travel_time"] = [round(sum(get_attr(row["route_travel_time"], "travel_time"))) for index, row in df.iterrows()]

In [22]:
oneway_len=[]
for index, row in df.iterrows():
    oneway = get_attr(row["route_travel_time"], "oneway")
    length = get_attr(row["route_travel_time"], "length")
    one_len = np.array(length)[np.array(oneway)]
    oneway_len.append(round(np.sum(one_len)))
    
df["oneway_length_by_travel_time"] = oneway_len

# Categorical to Numerical

In [23]:
ordinal_encoder = OrdinalEncoder()
object_cols = ["la_name_src","la_name_dst"]
df[["la_name_src_num","la_name_dst_num"]] = ordinal_encoder.fit_transform(df[object_cols])

# **Skewness**

In [24]:
df['area_src_reci'] =  1/df.area_src
df['area_dst_reci'] =  1/df.area_dst
df["oneway_length_sqrt"] = np.sqrt(df["oneway_length_by_travel_time"])

# Modelling

In [25]:
y = df.mean_travel_time
features = ['sourceid','dstid','dow','area_src_reci','area_dst_reci','fastest_travel_time','maxspeed_travel_time','geo_displacement_sqrt','oneway_length_sqrt','la_name_src_num','la_name_dst_num','direction']
X = df[features]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=123)

## XGBoost

In [26]:
xgb_params = {'learning_rate': 0.024704224090189218,
              'reg_lambda': 2.936733664947117,
              'reg_alpha': 57.40870727344214,
              'subsample': 0.6214411004415425,
              'colsample_bytree': 0.6251175460789683,
              'max_depth': 6}

xgb_model = XGBRegressor(n_estimators = 3642, random_state = 123,**xgb_params)

## Random Forest

In [27]:
rf_params = {'max_depth':165,
             'max_samples': 0.984425471209481,
             'max_features': 'auto',
             'min_samples_leaf': 1,
             'min_samples_split': 2,
             'n_estimators': 1832}

rf_model = RandomForestRegressor(**rf_params, random_state=123)

## Support Vector Regression

In [28]:
sv_params = {'degree': 5,
             'C': 489.52343725158937, 
             'coef0': 6.3906450532322845, 
             'tol': 0.9987114034413603, 
             'epsilon': 0.5708396421328326}

sv = SVR(kernel = 'poly',
                gamma = 'scale',
                shrinking = True,
                cache_size = 200,
                verbose = False,
                max_iter =-1,**sv_params)

## Ensembling

In [29]:
level0 = list()
level0.append(('xgb', xgb_model))
level0.append(('svr',sv))
level0.append(('rf',rf_model))

# define meta learner model
level1 = LinearRegression()
# define the stacking ensemble
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

model.fit(X_train,y_train)
predictions_ensemble = model.predict(X_valid)
rmse_ensemble = np.sqrt(metrics.mean_squared_error(y_valid,predictions_ensemble))

print(rmse_ensemble)

198.69126099248444


# Solving for testing dataset

In [30]:
ldn = london.drop(["DISPLAY_NAME","geometry","msoa_code","msoa_name","la_code","nodes_id_2","to_drop"],axis=1)

tsrc = test.merge(ldn, left_on = "sourceid", right_on = "MOVEMENT_ID", how = "left")
tsrc = tsrc.rename({"geoeast":"geoeast_src", "geonorth":"geonorth_src","popeast":"popeast_src","popnorth":"popnorth_src",
                  "la_name":"la_name_src","area_km2":"area_src", "msoa_code":"msoa_code_src", "x" : "src_x", "y" : "src_y",
                 "node_id_n" : "node_id_n_src", "node_id_e" : "node_id_e_src"},axis = 1)

tdst = tsrc.merge(ldn, left_on = "dstid", right_on = "MOVEMENT_ID", how = "left")
tdst = tdst.rename({"geoeast":"geoeast_dst", "geonorth":"geonorth_dst","popeast":"popeast_dst","popnorth":"popnorth_dst",
                  "la_name":"la_name_dst","area_km2":"area_dst","msoa_code":"msoa_code_dst","x" : "dst_x", "y" : "dst_y",
                  "node_id_n" : "node_id_n_dst", "node_id_e" : "node_id_e_dst"},axis = 1)

test = tdst.drop(["MOVEMENT_ID_x","MOVEMENT_ID_y"],axis=1)

In [31]:
test["geo_displacement"] = np.linalg.norm(test.loc[:, ["geoeast_src","geonorth_src"]].values - test.loc[:, ["geoeast_dst","geonorth_dst"]], axis=1)
test["geo_displacement_sqrt"] =  np.sqrt(test["geo_displacement"])

In [32]:
xDiffTest = test.loc[:, "geonorth_dst"].values - test.loc[:, "geonorth_src"].values
yDiffTest = test.loc[:, "geoeast_dst"].values - test.loc[:, "geoeast_src"].values
test["direction"] = np.round(np.degrees(np.arctan2(yDiffTest,xDiffTest)))

In [33]:
test["route_travel_time"] = get_route(test,weight="travel_time")

In [34]:
tmissing_route = test[test["route_travel_time"].isna()]
len(tmissing_route)

5

In [35]:
test.loc[tmissing_route.index,"route_travel_time"] = get_route(tmissing_route,"edges","travel_time")

In [36]:
tmissing_route = test[test["route_travel_time"].isna()]
len(tmissing_route)

0

In [37]:
test["maxspeed_travel_time"] = [round(np.mean(get_attr(row["route_travel_time"], "speed_kph"))) for index, row in test.iterrows()]
test["fastest_travel_time"] = [round(sum(get_attr(row["route_travel_time"], "travel_time"))) for index, row in test.iterrows()]

In [38]:
oneway_len_travel_time=[]
for index, row in test.iterrows():
    oneway = get_attr(row["route_travel_time"], "oneway")
    length = get_attr(row["route_travel_time"], "length")
    one_len = np.array(length)[np.array(oneway)]
    oneway_len_travel_time.append(round(np.sum(one_len)))
    
test["oneway_length_by_travel_time"] = oneway_len_travel_time


In [39]:
test['area_src_reci'] =  1/test.area_src
test['area_dst_reci'] =  1/test.area_dst
test["oneway_length_sqrt"] = np.sqrt(test["oneway_length_by_travel_time"])

In [40]:
test[["la_name_src_num","la_name_dst_num"]] = ordinal_encoder.fit_transform(test[object_cols])

In [41]:
test_data = test[features]
test_preds = model.predict(test_data)

In [42]:
output = pd.DataFrame({'sourceid': test.sourceid,
                       'dstid': test.dstid,
                       'dow':test.dow,
                       'predicted_mean_travel_time':test_preds})

output.to_csv('Final_submission3.csv', index=False)

In [43]:
output

,sourceid,dstid,dow,predicted_mean_travel_time
0,10,950,2,861.060918
1,10,889,2,572.303803
2,260,145,2,1022.129473
3,260,932,7,1373.880369
4,41,808,2,1152.060452
...,...,...,...,...
1957,712,435,7,1384.920442
1958,200,356,1,2600.342567
1959,200,716,3,395.489532
1960,657,549,5,1136.250902
